# MNIST for ML Beginners

- This tutorial goes through the code in [`mnist_softmax.py`](https://github.com/tensorflow/tensorflow/blob/v0.10.0/tensorflow/examples/tutorials/mnist/mnist_softmax.py) line by line.

In [3]:
# Download the MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


- The MNIST data is split into three parts:
    - 55,000 data points of training data (mnist.train)
    - 10,000 points of test data (mnist.test)
    - 5,000 points of validation data (mnist.validation)

- [Visualizing the MNIST data](http://colah.github.io/posts/2014-10-Visualizing-MNIST/)

- Each MNIST image is a 28 x 28 pixel image of a handwritten digit.
- We will be flattening the image into a vector (or "tensor") of 784 dimensions, each dimension representing a pixel's intensity (a value between 0 and 1).
- Since there are 55K images, our training data will have the shape [55000, 784].
- Each image has a corresponding label from 0 to 9.
- We will represent our labels in "one-hot" encoding, where each label will be a 10-dimension vector with one dimension "switched on", e.g. 3 would be [0, 0, 0, 1, 0, 0, 0, 0, 0, 0].
- Our training labels will have the shape [55000, 10].